In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [51]:
base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start={}&ref_=adv_nxt"
results = []

for start in range(1, 676, 50):
    url = base_url.format(start)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        movies = soup.find_all('div', class_='lister-item-content')

        for movie in movies:
            title = movie.find('a').text.strip()
            year = movie.find('span', class_='lister-item-year text-muted unbold').text.strip() if movie.find('span', class_='lister-item-year text-muted unbold') else 'Not specified'
            genre = movie.find('span', class_='genre').text.strip() if movie.find('span', class_='genre') else 'Not specified'
            director = movie.find('p', class_='').find_all('a')[0].text.strip() if movie.find('p', class_='') and movie.find('p', class_='').find_all('a') else 'Not specified'
            rating = movie.find('strong').text.strip()
            stars = ', '.join([star.text.strip() for star in movie.find('p', class_='').find_all('a')[1:]]) if movie.find('p', class_='') else 'Not specified'
            votes_revenue = movie.find_all('span', {'name': 'nv'})
            votes = votes_revenue[0].text.strip() if votes_revenue else 'Not specified'
            revenue = votes_revenue[1].text.strip() if len(votes_revenue) > 1 else 'Not specified'

            results.append({
                'title': title,
                'year': year,
                'genre': genre,
                'director': director,
                'rating': rating,
                'stars': stars,
                'votes': votes,
                'revenue': revenue
            })

        print(f'Scraped {start}-{start + 49}')
        time.sleep(1)  
    else:
        print(f'Error {response.status_code} for start={start}')

df = pd.DataFrame(results)
df.to_csv('movies.csv', index=False)


Scraped 1-50
Scraped 51-100
Scraped 101-150
Scraped 151-200
Scraped 201-250
Scraped 251-300
Scraped 301-350
Scraped 351-400
Scraped 401-450
Scraped 451-500
Scraped 501-550
Scraped 551-600
Scraped 601-650
Scraped 651-700


In [52]:
df.head()

,title,year,genre,director,rating,stars,votes,revenue
0,Goodfellas,(1990),"Biography, Crime, Drama",Martin Scorsese,8.7,"Robert De Niro, Ray Liotta, Joe Pesci, Lorrain...","1,178,923",$46.84M
1,The Silence of the Lambs,(1991),"Crime, Drama, Thriller",Jonathan Demme,8.6,"Jodie Foster, Anthony Hopkins, Lawrence A. Bon...","1,453,019",$130.74M
2,Terminator 2: Judgment Day,(1991),"Action, Sci-Fi",James Cameron,8.6,"Arnold Schwarzenegger, Linda Hamilton, Edward ...","1,113,245",$204.84M
3,Reservoir Dogs,(1992),"Crime, Thriller",Quentin Tarantino,8.3,"Harvey Keitel, Tim Roth, Michael Madsen, Chris...","1,033,347",$2.83M
4,My Cousin Vinny,(1992),"Comedy, Crime",Jonathan Lynn,7.6,"Joe Pesci, Marisa Tomei, Ralph Macchio, Mitche...","129,642",$52.93M


In [54]:
df.isnull().sum()

title       0
year        0
genre       0
director    0
rating      0
stars       0
votes       0
revenue     0
dtype: int64